In [ ]:
import os
# Configurar la variable ANTES de cualquier import de TF/Keras
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import tensorflow as tf
import tf_keras # Importante para confirmar instalación

# Verificación de Diagnóstico
import keras
print(f"Versión de Keras cargada: {keras.__version__}")

if not keras.__version__.startswith("2"):
    print("Sigue en Keras 3. Reinicia el entorno nuevamente.")
else:
    print(" Keras 2 (Legacy). El código de TF Hub funcionará.")

Versión de Keras cargada: 3.10.0
❌ ERROR CRÍTICO: Sigues en Keras 3. Reinicia el entorno nuevamente.


In [ ]:
# Instala la herramienta Consola del archivo ISIC
!pip install isic-cli #Sacada desde el repositorio de Github https://github.com/ImageMarkup/isic-cli/blob/master/README.md

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.2 MB/s eta 0:00:00


In [ ]:
#Descargar Imagenes + Metadatos
!isic image download --search "" --collections "249" --limit 0 BCN20000/ # Imagenes y Metadatos de BCN20000

#Comando extraido desde la misma ISIC ARCHIVE https://api.isic-archive.com/images/?query=&collections=249

If you have been granted special permissions, logging in with `isic user login` might return more data.


Successfully downloaded 18,946 images to BCN20000/.
Successfully wrote 18,946 metadata records to BCN20000/metadata.csv.
Successfully wrote attributions to BCN20000/attribution.txt.
Successfully wrote 1 license(s) to BCN20000/licenses.


In [ ]:
import csv

def cargar_metadatos(archivo):

    datos = {
        'isic_id': [],
        'diagnosis_1': []  # Las etiquetas serán 0 o 1
    }
    etiquetas_validas = {'Benign': 0, 'Malignant': 1}  # Mapeamos las etiquetas a 0 y 1

    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)  # Evita las columnas cabeceras.

            diagnosis_idx =7  # Cambiar al índice correspondiente si es necesario
            #7 para BCN20000
            for fila in reader:
                if len(fila) > diagnosis_idx:
                    diagnosis = fila[diagnosis_idx]
                    if diagnosis in etiquetas_validas:  # Solo almacenar Benign y Malignant
                        datos['isic_id'].append(fila[0])  # Almacena el ID de la imagen
                        datos['diagnosis_1'].append(etiquetas_validas[diagnosis])  # Asignamos la etiqueta 0 o 1

        print(f"Datos extraídos y filtrados correctamente. Se encontraron {len(datos['isic_id'])} registros válidos.")
        return datos

    except FileNotFoundError:
        print(f"Error: Archivo {archivo} no encontrado")
        return None

    except Exception as e:
        print(f"Error al procesar el archivo: {e}")
        return None

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def organizar_imagenes_en_carpetas(carpeta_origen, metadatos, carpeta_destino_base):

    # Crear las carpetas de destino
    for split in ['train', 'test']:
        for class_name in ['benign', 'malignant']:
            os.makedirs(os.path.join(carpeta_destino_base, split, class_name), exist_ok=True)

    # Separar los IDs por clase
    ids_benignos = []
    ids_malignos = []
    for id_img, etiqueta in zip(metadatos['isic_id'], metadatos['diagnosis_1']):
        if etiqueta == 0:  # Benign
            ids_benignos.append(id_img)
        elif etiqueta == 1: # Malignant
            ids_malignos.append(id_img)

    # Divide los IDs en conjuntos de entrenamiento y prueba para cada clase
    ids_benignos_train, ids_benignos_test = train_test_split(ids_benignos, test_size=0.3, random_state=42)
    ids_malignos_train, ids_malignos_test = train_test_split(ids_malignos, test_size=0.3, random_state=42)

    # Función auxiliar para copiar archivos
    def copiar_archivos(ids, carpeta_destino):
        for id_img in tqdm(ids, desc=f'Copiando a {os.path.basename(carpeta_destino)}'):
            nombre_archivo = f"{id_img}.jpg"
            ruta_origen = os.path.join(carpeta_origen, nombre_archivo)
            ruta_destino = os.path.join(carpeta_destino, nombre_archivo)
            if os.path.exists(ruta_origen):
                shutil.copy(ruta_origen, ruta_destino)
            else:
                print(f"Advertencia: No se encontró el archivo {ruta_origen}")

    # Copiar los archivos a sus directorios finales
    print("Copiando imágenes de entrenamiento benignas...")
    copiar_archivos(ids_benignos_train, os.path.join(carpeta_destino_base, 'train', 'benign'))

    print("Copiando imágenes de entrenamiento malignas...")
    copiar_archivos(ids_malignos_train, os.path.join(carpeta_destino_base, 'train', 'malignant'))

    print("Copiando imágenes de prueba benignas...")
    copiar_archivos(ids_benignos_test, os.path.join(carpeta_destino_base, 'test', 'benign'))

    print("Copiando imágenes de prueba malignas...")
    copiar_archivos(ids_malignos_test, os.path.join(carpeta_destino_base, 'test', 'malignant'))

    print("\nImágenes organizadas correctamente en carpetas 'train' y 'test' sin fugas de datos.")

In [ ]:
import tensorflow_hub as hub
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input

def build_googlenet_benchmark_legacy(num_classes=2):
    # URL oficial de GoogleNet (Inception v1)
    module_url = "https://tfhub.dev/google/imagenet/inception_v1/feature_vector/5"

    input_tensor = Input(shape=(224, 224, 3), name='input_image')

    base_layer = hub.KerasLayer(module_url, trainable=False, name='inception_v1_backbone')

    x = base_layer(input_tensor)

    x = layers.Dropout(0.4)(x)
    output_tensor = layers.Dense(num_classes, activation='softmax', name='classifier_output')(x)

    model = models.Model(inputs=input_tensor, outputs=output_tensor, name='GoogleNet_Benchmark_Legacy')

    return model

model_v1 = build_googlenet_benchmark_legacy()
model_v1.summary()

Model: "GoogleNet_Benchmark_Legacy"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 inception_v1_backbone (Ker  (None, 1024)              5607184   
 asLayer)                                                        
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 classifier_output (Dense)   (None, 2)                 2050      
                                                                 
Total params: 5609234 (21.40 MB)
Trainable params: 2050 (8.01 KB)
Non-trainable params: 5607184 (21.39 MB)
_________________________________________________________________


In [ ]:
import os
import shutil

# Eliminar el directorio del dataset antiguo para evitar conflictos y asegurar una regeneración limpia
if os.path.exists('/content/dataset'):
    shutil.rmtree('/content/dataset')
    print("Directorio '/content/dataset' anterior eliminado.")
else:
    print("El directorio '/content/dataset' no existía, no fue necesaria ninguna limpieza.")

El directorio '/content/dataset' no existía, no fue necesaria ninguna limpieza.


In [ ]:
# Asumiendo que los metadatos ya han sido cargados correctamente
metadatos = cargar_metadatos("/content/BCN20000/metadata.csv")

Datos extraídos y filtrados correctamente. Se encontraron 16702 registros válidos.


In [ ]:
organizar_imagenes_en_carpetas('/content/BCN20000/', metadatos, '/content/dataset')

Copiando imágenes de entrenamiento benignas...


Copiando a benign: 100%|██████████| 5481/5481 [00:00<00:00, 5920.35it/s]


Copiando imágenes de entrenamiento malignas...


Copiando a malignant: 100%|██████████| 6209/6209 [00:03<00:00, 1580.71it/s]


Copiando imágenes de prueba benignas...


Copiando a benign: 100%|██████████| 2350/2350 [00:00<00:00, 3883.99it/s]


Copiando imágenes de prueba malignas...


Copiando a malignant: 100%|██████████| 2662/2662 [00:02<00:00, 1080.56it/s]


Imágenes organizadas correctamente en carpetas 'train' y 'test' sin fugas de datos.


REVISAR GENERADORES!!!!

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def crear_generadores(carpeta_destino_base, batch_size=128):
  train_datagen = ImageDataGenerator(rescale=1./255)
  # Generador de datos para validación
  test_datagen = ImageDataGenerator(rescale=1./255)

  # Crear generador para entrenamiento
  train_generator = train_datagen.flow_from_directory(
      os.path.join(carpeta_destino_base, 'train'),
      target_size=(224, 224),
      batch_size=batch_size, #32
      class_mode='sparse', # sparse_categorical_crossentropy
      shuffle=True
      )

  # Crear generador para validación
  validation_generator = test_datagen.flow_from_directory(
      os.path.join(carpeta_destino_base, 'test'),
      target_size=(224, 224),
      batch_size=batch_size,
      class_mode='sparse', # sparse_categorical_crossentropy
      shuffle=False # Correcto para validación
      )

  print("Generadores de datos creados.")
  return train_generator, validation_generator

train_generator, validation_generator = crear_generadores('/content/dataset')

Found 11690 images belonging to 2 classes.
Found 5012 images belonging to 2 classes.
Generadores de datos creados.


In [ ]:
model_v1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), #
    loss='sparse_categorical_crossentropy', # O 'categorical' si usas one-hot encoding
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import CSVLogger
csv_logger_TL = CSVLogger('log_model_TL.csv', separator=',', append=False)

In [ ]:
callbacks_TL = [
    csv_logger_TL, # Logger
]

print("\n--- Iniciando Entrenamiento (Transfer Learning) ---")

history = model_v1.fit(
    train_generator,
    epochs=25, #Según Paper
    validation_data=validation_generator,
    callbacks=callbacks_TL
)

print("--- Entrenamiento (Transfer Learning) Finalizado ---")


--- Iniciando Entrenamiento (Transfer Learning) ---
Epoch 1/25
366/366 [==============================] - 240s 634ms/step - loss: 0.7542 - accuracy: 0.5395 - val_loss: 0.6353 - val_accuracy: 0.6443
Epoch 2/25
366/366 [==============================] - 226s 617ms/step - loss: 0.6809 - accuracy: 0.6133 - val_loss: 0.6129 - val_accuracy: 0.6706
Epoch 3/25
366/366 [==============================] - 226s 618ms/step - loss: 0.6458 - accuracy: 0.6405 - val_loss: 0.5908 - val_accuracy: 0.6866
Epoch 4/25
366/366 [==============================] - 232s 634ms/step - loss: 0.6365 - accuracy: 0.6507 - val_loss: 0.5856 - val_accuracy: 0.6923
Epoch 5/25
366/366 [==============================] - 233s 638ms/step - loss: 0.6253 - accuracy: 0.6631 - val_loss: 0.5789 - val_accuracy: 0.7005
Epoch 6/25
366/366 [==============================] - 235s 641ms/step - loss: 0.6163 - accuracy: 0.6682 - val_loss: 0.5761 - val_accuracy: 0.7019
Epoch 7/25
366/366 [==============================] - 224s 612ms/step -

In [ ]:
# Guardar el estado final absoluto
model_v1.save('final_googlenet_baseline.keras')
print("Modelo final guardado.")
from google.colab import files

# Nombre del archivo que definimos en el checkpoint
filename = 'final_googlenet_baseline.keras'

files.download(filename)

Modelo final guardado.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os

# Crear la carpeta para guardar los modelos si no existe
os.makedirs('/content/MisModelos', exist_ok=True)
model_v1.save('/content/MisModelos/model_v1.h5')
print("Modelos guardados exitosamente en /content/MisModelos/")

Modelos guardados exitosamente en /content/MisModelos/


/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import files
import os

# Ruta del modelo específico que se guardó en la celda anterior
modelo_path = '/content/MisModelos/model_v1.h5'

print(f"Intentando descargar el modelo: {os.path.basename(modelo_path)}")

# Verificar si el archivo del modelo existe antes de iniciar la descarga
if os.path.exists(modelo_path):
    files.download(modelo_path)
    print("\nDescarga completada exitosamente.")
else:
    print(f"\nError: No se encontró el archivo del modelo en la ruta especificada: {modelo_path}")
    print("Por favor, asegúrate de que la celda anterior para guardar el modelo se haya ejecutado sin errores.")

Intentando descargar el modelo: model_v1.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Descarga completada exitosamente.


In [ ]:
import pandas as pd
import os

def convertir_csv_a_excel(ruta_csv, ruta_excel):

    if not os.path.exists(ruta_csv):
        print(f"Advertencia: El archivo CSV no fue encontrado en la ruta: {ruta_csv}")
        return False

    try:
        # Leer el archivo CSV con pandas
        df = pd.read_csv(ruta_csv)

        # Guardar el DataFrame como archivo Excel
        df.to_excel(ruta_excel, index=False)

        print(f"Métricas de '{ruta_csv}' guardadas exitosamente en: {ruta_excel}")
        return True

    except Exception as e:
        print(f"Error durante la conversión de '{ruta_csv}' a Excel: {e}")
        return False

In [ ]:
from google.colab import files

# Lista de los archivos CSV de registro que se generaron
archivos_csv_registros = [
    'log_model_TL.csv'
]

# Carpeta para guardar los archivos Excel
carpeta_excel = '/content/Metricas_Excel/'
os.makedirs(carpeta_excel, exist_ok=True)

print("--- Iniciando conversión y descarga de métricas ---")
for nombre_csv in archivos_csv_registros:
    nombre_excel = nombre_csv.replace('.csv', '.xlsx')
    ruta_excel_destino = os.path.join(carpeta_excel, nombre_excel)

    # Intentar convertir el archivo
    if convertir_csv_a_excel(nombre_csv, ruta_excel_destino):
        # Si la conversión fue exitosa, descargar el archivo
        print(f"Iniciando descarga de {ruta_excel_destino}...")
        files.download(ruta_excel_destino)

print("\n--- Proceso de conversión y descarga finalizado. ---")

--- Iniciando conversión y descarga de métricas ---
Métricas de 'log_model_TL.csv' guardadas exitosamente en: /content/Metricas_Excel/log_model_TL.xlsx
Iniciando descarga de /content/Metricas_Excel/log_model_TL.xlsx...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


--- Proceso de conversión y descarga finalizado. ---
